In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('enhanced_tweets_informations.csv', encoding='latin-1', names=['target', 'id', 'date', 'flag', 'user', 'text', 'tweet_length_chars', 'tweet_length_words', 'mention_only', 'unreadable', 'too_many_numbers'])

/var/folders/76/4bn4ybn13vq8nd277yr2xdk40000gp/T/ipykernel_23764/1796006953.py:1: DtypeWarning: Columns (0,1,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('enhanced_tweets_informations.csv', encoding='latin-1', names=['target', 'id', 'date', 'flag', 'user', 'text', 'tweet_length_chars', 'tweet_length_words', 'mention_only', 'unreadable', 'too_many_numbers'])


In [6]:
print(f"Number of rows: {len(df)}")


Number of rows: 1600001


In [7]:
df = df[(df['mention_only'] != 1) & (df['unreadable'] != 1) & (df['too_many_numbers'] != 1) & (df['is_duplicated'] != 1)]


In [8]:
print(f"Number of rows: {len(df)}")


Number of rows: 1595327


In [9]:
df = df.drop(columns=['flag'])


In [11]:
df = df.sample(n=1000, random_state=42)


In [10]:
import re
import os
import requests
from symspellpy import SymSpell, Verbosity

# Initialize SymSpell
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Download and load the frequency dictionary if not already available
dict_path = "frequency_dictionary_en_82_765.txt"
if not os.path.exists(dict_path):
    url = "https://raw.githubusercontent.com/mammothb/symspellpy/master/symspellpy/data/frequency_dictionary_en_82_765.txt"
    response = requests.get(url)
    with open(dict_path, "wb") as f:
        f.write(response.content)

# Load the dictionary into SymSpell
sym_spell.load_dictionary(dict_path, term_index=0, count_index=1)

def remove_repeated_chars(word):
    """Reduces any letter repeated 2+ times to a single occurrence."""
    return re.sub(r'(\w)\1{2,}', r'\1', word)

def correct_text(text):
    """Fixes repeated characters and corrects misspellings using SymSpell."""
    words = text.split()
    cleaned_words = [remove_repeated_chars(word) for word in words]
    
    corrected_words = []
    for word in cleaned_words:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        corrected_word = suggestions[0].term if suggestions else word  # Use the best suggestion
        corrected_words.append(corrected_word)

    return " ".join(corrected_words)

# Apply correction to the 'text' column in the DataFrame
df['text'] = df['text'].apply(correct_text)

# Show 5 examples of modified text
for i, row in df.head(5).iterrows():
    original_text = row['text']
    modified_text = correct_text(original_text)
    print(f"Original: {original_text}\nModified: {modified_text}\n")


Oh my god what are you doing?


/Users/anne-soline.guilbert-ly@dataiku.com/EDF-sentiment-analysis/.venv/lib/python3.9/site-packages/symspellpy/symspellpy.py:1137: UserWarning: Failed to parse frequency count as a 64 bit integer.
  warnings.warn("Failed to parse frequency count as a 64 bit integer.")
